# Assignment 1 : Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [103]:
# Rohan Chaudhry rc43755 

#You may add additional import if you want
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import tree 
import time

In [104]:
%matplotlib inline

In [105]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1
5,1,1,44,43,41,41,37,29,28.356400,6.935636,2.305771,0.323724,0.000000,0.000000,0.000000,0.000000,0.502831,0.126741,0,1
6,1,0,29,29,29,27,25,16,15.448398,9.113819,1.633493,0.000000,0.000000,0.000000,0.000000,0.000000,0.541743,0.139575,0,1
7,1,1,6,6,6,6,2,1,20.679649,9.497786,1.223660,0.150382,0.000000,0.000000,0.000000,0.000000,0.576318,0.071071,1,0
8,1,1,22,21,18,15,13,10,66.691933,23.545543,6.151117,0.496372,0.000000,0.000000,0.000000,0.000000,0.500073,0.116793,0,1
9,1,1,79,75,73,71,64,47,22.141784,10.054384,0.874633,0.099780,0.023386,0.000000,0.000000,0.000000,0.560959,0.109134,0,1


### 1. Data preprocessing  & dimensionality reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames - one that contains all of the feature values and one that contains the class labels.

In [106]:
# Predict whether or not image contains DR 
# class label == what we want to solve for ... 19 
# features --> 1-18; 

df = data # copy of the dataset 
features = df[['quality', 'prescreen','ma2','ma3','ma4','ma5','ma6','ma7','exudate8','exudate9','exudate10','exudate11','exudate12','exudate13','exudate14','exudate15','euDist','diameter','amfm_class']]
classes = df[['label']]


Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). 

Only standardize the the features, not the class labels! This will be required for running the principal component analysis (PCA) dimensionality reduction later. Note that `StandardScaler` returns a numpy array.

In [107]:
#scaled_features = StandardScaler(features)
scaled_features = features 
col_names = ['quality', 'prescreen','ma2','ma3','ma4','ma5','ma6','ma7','exudate8','exudate9','exudate10','exudate11','exudate12','exudate13','exudate14','exudate15','euDist','diameter','amfm_class']

''' 
feats = scaled_features[col_names]
scaler = StandardScaler().fit(feats.values)
feats = scaler.transform(feats.values) 
#scaled_features = StandardScaler.fit_transform(features.values) 
scaled_features[col_names] = feats 
print(scaled_features)
# type is <class 'pandas.core.frame.DataFrame'>
'''

scaled_features = StandardScaler().fit_transform(scaled_features)
print(type(scaled_features))
print(scaled_features.shape)
print(scaled_features[0])
print("mean: ", scaled_features.mean())
print("std: ", scaled_features.std())



<class 'numpy.ndarray'>
(1151, 19)
[ 0.05905386  0.2982129  -0.6414863  -0.61878225 -0.57646257 -0.63002889
 -0.55111605 -0.47374452 -0.24291735 -0.24600298 -0.29696618 -0.2715091
 -0.21832427 -0.19440942 -0.20512449 -0.18616904 -1.29476283 -0.46865568
  1.40504812]
mean:  -2.8876256637559514e-17
std:  1.0


Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [108]:
# 80% training, 20% testing 
#x_train, x_test = train_test_split(scaled_features,train_size=.8,test_size=.2)

X_train, X_test, y_train, y_test = train_test_split(scaled_features, classes, test_size=0.20, random_state=None)

print(type(x_train))

print("X_train=",X_train.shape, " X_test=", X_test.shape)
print("y_train=",y_train.shape, " y_test=", y_test.shape)




print('\n\n\n') 
'''
print('\nTraining Set:')
print('Training shape: ' + str(x_train.shape))
print('Total number of rows: '+ str(x_train.shape[0]))
print('Total number of records (num rows*num cols) = ' + str(x_train.shape[0]*x_train.shape[1]))
print('\n\nTesting Set:')
print('Testing shape: ' + str(x_test.shape)) 
print('Total number of rows: '+ str(x_test.shape[0]))
print('Total number of records (num rows*num cols) = ' + str(x_test.shape[0]*x_test.shape[1]))
'''

<class 'numpy.ndarray'>
X_train= (920, 19)  X_test= (231, 19)
y_train= (920, 1)  y_test= (231, 1)






"\nprint('\nTraining Set:')\nprint('Training shape: ' + str(x_train.shape))\nprint('Total number of rows: '+ str(x_train.shape[0]))\nprint('Total number of records (num rows*num cols) = ' + str(x_train.shape[0]*x_train.shape[1]))\nprint('\n\nTesting Set:')\nprint('Testing shape: ' + str(x_test.shape)) \nprint('Total number of rows: '+ str(x_test.shape[0]))\nprint('Total number of records (num rows*num cols) = ' + str(x_test.shape[0]*x_test.shape[1]))\n"

Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [109]:
# PCA on training set 
# imported PCA at the very top 

# pca = PCA(n_components = 18)
# pca = PCA(.95) # 95% accuracy # had originally 


pca = PCA()
pca_data = pca.fit_transform(X_train)


var_exp = pca.explained_variance_ratio_
print("Variance in each PC: ", var_exp,'\n')

cum_var_exp = np.cumsum(var_exp)
print("Cumulative variance: ", cum_var_exp,'\n')

n_cols = 1 + np.argmax(cum_var_exp > 0.95)
print("n_cols to keep:", n_cols,'\n')

#reduce training set to size
X_train_pca = pca_data[:, :n_cols] # all rows, columns 0 to n_cols
X_test_pca = pca.transform(X_test)[:, :n_cols] 
print (X_train_pca.shape, X_test_pca.shape) #they've both been reduced to 2 dimensions

print('\n'*5)

# pca = PCA
# pca = PCA(n_components = 'mle') # guesses number of features 
#x_train_pca = PCA(x_train)
#print(x_train_pca)
'''
pca.fit(x_train)

# PCA variance ratio 
print()
print(pca.explained_variance_ratio_)  
print(1+pca.n_components_)

# reduce PCA set 
new = PCA(n_components=9)
mainComp = new.fit_transform(x_train) # main components used 
mainDf = pd.DataFrame(data = mainComp, columns = ['Main1','Main2','Main3','Main4','Main5','Main6','Main7','Main8','Main9'])
#finalDf = pd.concat([mainDf, df[['target']]], axis =1)
print(mainDf)
'''



# set transform 

Variance in each PC:  [3.19566304e-01 2.61654065e-01 1.04690152e-01 6.57606662e-02
 5.97959488e-02 4.91431913e-02 4.42156630e-02 3.83508996e-02
 2.50749013e-02 1.22690101e-02 7.75748321e-03 5.86494281e-03
 2.34904263e-03 1.23863260e-03 1.02588147e-03 8.31163424e-04
 2.28062810e-04 1.22192105e-04 6.17984758e-05] 

Cumulative variance:  [0.3195663  0.58122037 0.68591052 0.75167119 0.81146714 0.86061033
 0.90482599 0.94317689 0.96825179 0.9805208  0.98827828 0.99414323
 0.99649227 0.9977309  0.99875678 0.99958795 0.99981601 0.9999382
 1.        ] 

n_cols to keep: 9 

(920, 9) (231, 9)








"\npca.fit(x_train)\n\n# PCA variance ratio \nprint()\nprint(pca.explained_variance_ratio_)  \nprint(1+pca.n_components_)\n\n# reduce PCA set \nnew = PCA(n_components=9)\nmainComp = new.fit_transform(x_train) # main components used \nmainDf = pd.DataFrame(data = mainComp, columns = ['Main1','Main2','Main3','Main4','Main5','Main6','Main7','Main8','Main9'])\n#finalDf = pd.concat([mainDf, df[['target']]], axis =1)\nprint(mainDf)\n"

### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [110]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)
# Train it
# clf = clf.fit(features, classes)
# X_train, X_test, y_train, y_test
# X_train_pca , X_train_pca 

#clf = clf.fit(X_train, y_train)
clf = clf.fit(X_train_pca, y_train)

# The trained tree is now stored in the variable 'clf'

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [111]:
pred_Y = clf.predict(X_test_pca)
#print(pred_Y)

from sklearn.metrics import accuracy_score

print('Accuracy on test data is:', (accuracy_score(y_test, pred_Y))*100)


Accuracy on test data is: 65.36796536796537


Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [112]:
# your code goes here

### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q7. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the data before it underwent PCA. So you want to use the standardized data that came out of Q2. 

In [113]:
from sklearn.model_selection import cross_val_score
#print(type(X_train))
#print(type(y_train))
''' 
#make numpy arrays 
X_train_vals = X_train.values
y_train_vals = y_train.values
#print(type(X_train_vals))

print(scaled_features.shape)
print(y_train.shape)
''' 

#idk = cross_val_score(scaled_features,X_train_vals, y_train_vals,cv=10)
# idk = cross_val_score(clf,scaled_features, y_train,cv=10)
idk = cross_val_score(clf,X_train, y_train,cv=10)
print("Accuracy: ", idk.mean()*100)

Accuracy:  59.582363639165884


Q8. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [114]:
from sklearn.model_selection import GridSearchCV

# Try max_depth, min_samples_leaf values of 5,10,15,20
# Try max_features values of 5,10,15
params = {"max_depth": [5,10,15,20],
          "min_samples_leaf": [5,10,15,20],
          "max_features": [5,10,15]}


# Create the GridSearchCV object:
# Pass in our classifier (the decision tree), the parameters to test,
# how many folds of CV we want, and what scoring metric to use
grid_search = GridSearchCV(clf, params, cv=5, scoring='accuracy')

# Kick it off... start training classifiers with the different max_depths specified
# A 5-fold CV will be run for each max_depth and accuracy will be assesed for each max_depth
grid_search.fit(features, classes)

# What did it find as the best max_depth?
print(grid_search.best_params_)
# What was the accuracy at this best max_depth?
print("Accuracy:", grid_search.best_score_*100)


{'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 10}
Accuracy: 65.07384882710686


Q9. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [115]:
nested_score = cross_val_score(grid_search, features, classes, cv=3)

print("Accuracy:", nested_score.mean()*100)

Accuracy: 63.59470191470845


Q10. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What can we do to improve the accuracy of the model? Answer as a comment:

In [116]:
'''
Potential ways to improve the model's accuracy include: 
- involving more data in the test/train sets 
- scrubbing the data for outliers and ensuring a clean set 
- feature transformation for all features into a normalized 0-1 scale
- improving our feature selection 
'''

"\nPotential ways to improve the model's accuracy include: \n- involving more data in the test/train sets \n- scrubbing the data for outliers and ensuring a clean set \n- feature transformation for all features into a normalized 0-1 scale\n- improving our feature selection \n"